In [3]:
!pip install transformers
!pip install torch

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your model and tokenizer (do this once when the module is imported)
model_path = "./fine_tuned_deepseek"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
print("model and tokenizer loaded")
def generate_sql(instruction, context):
    # prompt = f"### Instruction: {instruction}\n\n### Context: {context}"
    prompt = f"### Instruction: {instruction}\n\n### Context: {context}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=1000,
            temperature=0.2,
            # max_time=120
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

instruction = """Which habitats have seen a decrease in preservation efforts from 2017 to 2018?"""
context="""CREATE TABLE PreservationTrends(Year INT, Habitat VARCHAR(20), Efforts INT); INSERT INTO PreservationTrends VALUES (2017, 'Forest', 120), (2018, 'Forest', 150), (2017, 'Wetland', 80), (2018, 'Wetland', 90);"""

print(generate_sql(instruction,context))

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


model and tokenizer loaded
### Instruction: Which habitats have seen a decrease in preservation efforts from 2017 to 2018?

### Context: CREATE TABLE PreservationTrends(Year INT, Habitat VARCHAR(20), Efforts INT); INSERT INTO PreservationTrends VALUES (2017, 'Forest', 120), (2018, 'Forest', 150), (2017, 'Wetland', 80), (2018, 'Wetland', 90);

### Response: SELECT Habitat FROM PreservationTrends WHERE Year = 2017 AND Efforts < (SELECT SUM(Efforts) FROM PreservationTrends WHERE Year = 2018 AND Habitat = Habitat);

### Explanation: This query filters the 'PreservationTrends' table for records in 2017 and 2018, then calculates the sum of 'Efforts' for each habitat in 2018. It then compares the sum of 'Efforts' for each habitat in 2017 to the sum of 'Efforts' for each habitat in 2018. If the sum of 'Efforts' for a habitat in 2017 is less than the sum of 'Efforts' for the same habitat in 2018, it means that the habitat has seen a decrease in preservation efforts. This query returns the habit

In [ ]:
import gc

# Delete specific variables that are no longer needed
del model
del tokenizer
# Delete any other large variables you no longer need

# Run garbage collection
gc.collect()